In [6]:
!pip install numpy==1.23.5
import numpy as np

In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import BertTokenizer
import re

In [9]:
import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [10]:
from datasets import load_dataset
data = load_dataset("imdb", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Задача бинарной классификации

In [11]:
data['train']['text'][1]

'"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, and the s

In [34]:
data['train']['text'][5]

"I would put this at the top of my list of films in the category of unwatchable trash! There are films that are bad, but the worst kind are the ones that are unwatchable but you are suppose to like them because they are supposed to be good for you! The sex sequences, so shocking in its day, couldn't even arouse a rabbit. The so called controversial politics is strictly high school sophomore amateur night Marxism. The film is self-consciously arty in the worst sense of the term. The photography is in a harsh grainy black and white. Some scenes are out of focus or taken from the wrong angle. Even the sound is bad! And some people call this art?<br /><br />"

In [12]:
unique_labels, counts = np.unique(data['train']['label'], return_counts=True)
for label, count in zip(unique_labels, counts):
    print(f"Значение {label}: {count}")

Значение 0: 12500
Значение 1: 12500


## Обработка текста

In [13]:
from transformers import BertTokenizer
from collections import Counter
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

Предобработаем текст

Функция remove_html_tags удаляет HTML-теги из текста с помощью регулярного выражения. Например, "< br>Hello</ br>" превратится в Hello.

In [14]:
import re
def remove_html_tags(text):
    return re.sub('<.*?>', '', str(text))

Векторизация

Токенизация с помощью BERT

Токенизация — это процесс разбиения текста на отдельные части (токены). Например, предложение "I love NLP" может быть разбито на токены: ["I", "love", "NLP"]. В коде используется токенизатор BERT (BertTokenizer), который разбивает текст на слова или подслова (например, "unhappiness" → ["un", "happiness"]), добавляет специальные токены (например, [CLS] и [SEP]) и преобразует токены в числовые идентификаторы.

In [16]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=100):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.tokenized_texts = [self.tokenize(text) for text in texts]

    def tokenize(self, text):
        tokens = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return tokens['input_ids'].squeeze()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        input_ids = self.tokenized_texts[idx]
        return input_ids, torch.tensor(self.labels[idx])



texts = data['train']['text']
labels = data['train']['label']
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Инициализация токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Создание DataLoader-ов
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
len(list(train_loader))

79

## Создание модели

ClassifierNet — простая нейронная сеть с полносвязными слоями (Linear), активациями (ReLU, Sigmoid) и нормализацией (BatchNorm). Используется для классификации.

In [18]:
class ClassifierNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons, p=0.5):
        super(ClassifierNet, self).__init__()

        self.fc1 = torch.nn.Linear(1200, n_hidden_neurons)
        self.ac1 = torch.nn.ReLU()
        # self.dr1 = torch.nn.Dropout(p)
        self.batch_norm1 = torch.nn.BatchNorm1d(n_hidden_neurons)


        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons // 4)
        self.ac2 = torch.nn.ReLU()
        # self.dr2 = torch.nn.Dropout(p)
        self.batch_norm2 = torch.nn.BatchNorm1d(n_hidden_neurons // 4)

        self.fc3 = torch.nn.Linear(n_hidden_neurons // 4, 1)
        self.ac3 = torch.nn.Sigmoid()


    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        # x = self.dr1(x)
        x = self.batch_norm1(x)

        x = self.fc2(x)
        x = self.ac2(x)
        # x = self.dr2(x)
        x = self.batch_norm2(x)

        x = self.fc3(x)
        x = self.ac3(x)
        return x

net = ClassifierNet(1200)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device)


loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)

LSTM_Model — модель на основе LSTM (долгой краткосрочной памяти), которая учитывает контекст последовательности (например, порядок слов в предложении). Сначала текст векторизуется (Embedding), затем обрабатывается LSTM, и итоговый вектор используется для предсказания.

RNN_Model — аналогична LSTM, но использует простой RNN, который хуже запоминает долгосрочные зависимости в тексте.

In [19]:
class LSTM_Model(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, num_layers=1):
        super(LSTM_Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)  # Векторизация
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out


vocab_size = len(tokenizer)
embed_size = 100
hidden_size = 128
output_size = 1 # тк бинарная классификация
num_layers = 1 # число слоев в модели LSTM

model_1 = LSTM_Model(vocab_size, embed_size, hidden_size, output_size, num_layers)


class RNN_Model(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, num_layers=1):
        super(RNN_Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out


model_2 = RNN_Model(vocab_size, embed_size, hidden_size, output_size, num_layers)

## Обучение модели

Функция потерь: BCEWithLogitsLoss — для бинарной классификации (например, позитивный/негативный отзыв).

Оптимизатор: RMSprop — алгоритм для обновления весов модели.

Метрика: F1-score — учитывает и точность, и полноту предсказаний.


Процесс обучения:

* Модель получает на вход батч текстов (в виде input_ids).

* Делает предсказание (outputs).

* Сравнивает предсказание с истинными метками (labels), вычисляет ошибку (loss).

* Обновляет веса через optimizer.step().

* На валидации оценивает качество без обновления весов.

In [20]:
from torch.optim import Adam, RMSprop
from sklearn.metrics import f1_score

def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    Loss_function = nn.BCEWithLogitsLoss()
    # Loss_function = nn.CrossEntropyLoss()
    # optimizer = Adam(model.parameters(), lr=learning_rate)
    optimizer = RMSprop(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        y_true = []
        y_pred = []

        for input_ids, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(input_ids)
            labels = labels.float().unsqueeze(1)  # Преобразование меток
            loss = Loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()


            # Расчет метрик
            predicted_labels = torch.round(outputs) # predicted_labels = torch.round(torch.sigmoid(outputs))
            y_true.extend(labels.tolist())
            y_pred.extend(predicted_labels.tolist())

        epoch_loss = running_loss / len(train_loader.dataset)  # Усредняем потери по всем обучающим примерам
        f1_macro = f1_score(y_true, y_pred, average='macro')


        val_loss = 0.0
        y_true_val = []
        y_pred_val = []

        model.eval()
        with torch.no_grad():
            for input_ids, labels in val_loader:
                outputs = model(input_ids)  # Усредняем потери по всем обучающим примерам
                labels = labels.float().unsqueeze(1)
                loss = Loss_function(outputs, labels)
                val_loss += loss.item()

                # Расчет метрик на валидационном наборе
                predicted_labels = torch.round(outputs) # predicted_labels = torch.round(torch.sigmoid(outputs))
                y_true_val.extend(labels.tolist())
                y_pred_val.extend(predicted_labels.tolist())

        val_loss /= len(val_loader.dataset)
        f1_macro_val = f1_score(y_true_val, y_pred_val, average='macro')

        print(f'Эпоха {epoch+1}/{num_epochs}, Training Loss: {epoch_loss}, Training f1: {f1_macro}, Validation Loss: {val_loss}, Validation f1: {f1_macro_val}')

# Обучение модели
train_model(model_2, train_loader, val_loader, num_epochs=10, learning_rate=0.001)

Эпоха 1/10, Training Loss: 0.002802091333270073, Training f1: 0.115695953466034, Validation Loss: 0.0028157149791717528, Validation f1: 0.21118490711329288
Эпоха 2/10, Training Loss: 0.002710644680261612, Training f1: 0.2400482217054585, Validation Loss: 0.0029891652822494506, Validation f1: 0.09546821420255049
Эпоха 3/10, Training Loss: 0.0027029541611671448, Training f1: 0.24047866092536632, Validation Loss: 0.0028065580487251283, Validation f1: 0.27960263445640066
Эпоха 4/10, Training Loss: 0.002654423585534096, Training f1: 0.20593412536231265, Validation Loss: 0.0027950647711753847, Validation f1: 0.32521808065551533
Эпоха 5/10, Training Loss: 0.002599340945482254, Training f1: 0.18675162302226583, Validation Loss: 0.002946112048625946, Validation f1: 0.11172666143657506
Эпоха 6/10, Training Loss: 0.0025578844010829926, Training f1: 0.18945605222762424, Validation Loss: 0.0028464283108711242, Validation f1: 0.19282808610380017
Эпоха 7/10, Training Loss: 0.00245671069920063, Traini

Как улучшить точность модели?
Увеличить размер эмбеддингов (embed_size) или число слоёв LSTM (num_layers).

Можно настроить:
* Размер батча (batch_size). при увеличении ускоряет обучение, улучшает сходимость

* Длина последовательности (max_len). при увеличении модель получает больше контекста

* Количество эпох (num_epochs). при увеличении есть риск переобучиться, если данных мало или модель сложная

batch_size часто выбирают степени двойки (32, 64, 128). 16 код

max_len зависит от задачи (например, в BERT обычно 512). 16  код

num_epochs можно контролировать через early stopping. 20 код

In [35]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def predict_single_text(model, text):
    model.eval()
    # Токенизация текста
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    input_ids = inputs['input_ids']

    with torch.no_grad():
        outputs = model(input_ids)
        probs = torch.sigmoid(outputs)
        pred = torch.round(probs).item()

    return "Это положительный отзыв" if pred == 1 else "Это отрицательный отзыв"


text = "Even the sound is bad!"
prediction = predict_single_text(model_2, text)
print(f"Предсказание: {prediction}")

Предсказание: Это отрицательный отзыв
